# Module 4: Classical ML Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NabKh/ML-for-Materials-Science/blob/main/Tutorial-07-ML-Discovery/notebooks/04_classical_ml_models.ipynb)

---

> **Before You Start:** Please check the [INSTALLATION_GUIDE.md](../../INSTALLATION_GUIDE.md) for setup instructions. For Google Colab:
> ```python
> !pip install pymatgen matminer shap -q
> ```
> Then restart the runtime (Runtime → Restart runtime).

---

## 🎯 Learning Objectives

1. **Train** various ML models: Linear, Ridge, Random Forest, XGBoost
2. **Understand** when to use each model type
3. **Compare** model performance systematically
4. **Avoid** common pitfalls in model training

---

**⏱️ Estimated time: 90 minutes** | **📚 Difficulty: 🟡 Intermediate**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Create figures directory
os.makedirs('figures', exist_ok=True)

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

try:
    from xgboost import XGBRegressor
    HAS_XGBOOST = True
except:
    HAS_XGBOOST = False
    print("XGBoost not installed, skipping XGB examples")

import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
print("✅ Libraries loaded!")

✅ Libraries loaded!


<cell_type>markdown</cell_type>## 1. The ML Model Landscape

### 📖 Theory

<div style="background: linear-gradient(135deg, #1e293b 0%, #0f172a 100%); padding: 20px; border-radius: 10px; border-left: 4px solid #6366f1;">

**Supervised Learning** aims to learn a function $f: X \rightarrow y$ that maps input features to target property.

The goal is to minimize a **loss function**:
$$\mathcal{L} = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2 + \lambda \cdot \text{Regularization}$$

where $\hat{y}_i = f(x_i)$ is the model prediction.

</div>

### Model Overview

| Model | Formula | Pros | Cons | Best For |
|-------|---------|------|------|----------|
| Linear | $\hat{y} = \mathbf{w}^T\mathbf{x} + b$ | Fast, interpretable | Linear only | Baselines |
| Ridge | $+ \lambda\|\|\mathbf{w}\|\|_2^2$ | Handles collinearity | Still linear | Many features |
| Lasso | $+ \lambda\|\|\mathbf{w}\|\|_1$ | Feature selection | Still linear | Sparse models |
| Random Forest | Ensemble of trees | Robust, non-linear | Less interpretable | General use |
| XGBoost | Gradient boosted trees | State-of-the-art | Many hyperparameters | Competitions |

### Mathematical Details

**Linear Regression** finds weights by minimizing squared error:
$$\mathbf{w}^* = \arg\min_{\mathbf{w}} \sum_{i=1}^{n}(y_i - \mathbf{w}^T\mathbf{x}_i)^2$$

Closed-form solution: $\mathbf{w}^* = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$

**Ridge Regression** adds L2 penalty to prevent large weights:
$$\mathbf{w}^* = \arg\min_{\mathbf{w}} \sum_{i=1}^{n}(y_i - \mathbf{w}^T\mathbf{x}_i)^2 + \alpha\sum_{j=1}^{p}w_j^2$$

**Lasso Regression** adds L1 penalty for sparse solutions:
$$\mathbf{w}^* = \arg\min_{\mathbf{w}} \sum_{i=1}^{n}(y_i - \mathbf{w}^T\mathbf{x}_i)^2 + \alpha\sum_{j=1}^{p}|w_j|$$

**Random Forest** averages predictions from $B$ decision trees:
$$\hat{y} = \frac{1}{B}\sum_{b=1}^{B}T_b(\mathbf{x})$$

Each tree $T_b$ is trained on a bootstrap sample with random feature subsets.

**Gradient Boosting** builds trees sequentially to correct errors:
$$F_m(\mathbf{x}) = F_{m-1}(\mathbf{x}) + \eta \cdot h_m(\mathbf{x})$$

where $h_m$ is fitted to the negative gradient (residuals) of the loss.

In [2]:
# Generate synthetic materials data for demonstration
np.random.seed(42)
n_samples = 500

# Features (imagine these are from matminer)
X = np.random.randn(n_samples, 20)

# Target: non-linear relationship (band gap)
y = (
    2.0 * X[:, 0] +                    # Linear term
    1.5 * X[:, 1]**2 +                 # Quadratic term
    0.8 * X[:, 2] * X[:, 3] +          # Interaction
    0.5 * np.sin(X[:, 4] * np.pi) +    # Non-linear
    np.random.randn(n_samples) * 0.5   # Noise
)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 400 samples
Test set: 100 samples


## 2. Model Training and Comparison

In [3]:
# Define models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Lasso (α=0.1)': Lasso(alpha=0.1),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
}

if HAS_XGBOOST:
    models['XGBoost'] = XGBRegressor(n_estimators=100, max_depth=5, random_state=42, verbosity=0)

# Train and evaluate all models
results = []

print("Training models...\n")
print(f"{'Model':<25} {'Train R²':>10} {'Test R²':>10} {'Test MAE':>10} {'Test RMSE':>10}")
print("="*70)

for name, model in models.items():
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    results.append({
        'Model': name,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'Test MAE': test_mae,
        'Test RMSE': test_rmse,
        'Gap': train_r2 - test_r2
    })
    
    # Flag overfitting
    flag = "⚠️" if (train_r2 - test_r2) > 0.1 else "✓"
    print(f"{name:<25} {train_r2:>10.4f} {test_r2:>10.4f} {test_mae:>10.4f} {test_rmse:>10.4f} {flag}")

results_df = pd.DataFrame(results).sort_values('Test R²', ascending=False)

Training models...

Model                       Train R²    Test R²   Test MAE  Test RMSE
Linear Regression             0.4550     0.4539     1.5874     2.3122 ✓
Ridge (α=1.0)                 0.4550     0.4538     1.5874     2.3125 ✓
Lasso (α=0.1)                 0.4419     0.4987     1.5416     2.2153 ✓
Decision Tree                 0.9857     0.7227     1.2368     1.6476 ⚠️
Random Forest                 0.9662     0.8322     0.9369     1.2817 ⚠️


Gradient Boosting             0.9974     0.8505     0.8555     1.2097 ⚠️
XGBoost                       1.0000     0.8442     0.8432     1.2350 ⚠️


In [ ]:
# Visualize results with professional styling
fig, axes = plt.subplots(1, 2, figsize=(15, 6), facecolor='white')

# Color palette - consistent with other figures
colors = {
    'train': '#0ea5e9',      # Sky blue
    'test': '#ec4899',       # Pink
    'text': '#1e293b',
    'grid': '#e2e8f0',
    'good': '#10b981',
    'warning': '#ef4444'     # Red for overfitting
}

# R² comparison
ax1 = axes[0]
x = np.arange(len(results_df))
width = 0.35

bars1 = ax1.bar(x - width/2, results_df['Train R²'], width, label='Train R²', 
                color=colors['train'], alpha=0.85, edgecolor='white', linewidth=1)
bars2 = ax1.bar(x + width/2, results_df['Test R²'], width, label='Test R²', 
                color=colors['test'], alpha=0.85, edgecolor='white', linewidth=1)

ax1.set_ylabel('R² Score', fontsize=12, color=colors['text'])
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold', color=colors['text'])
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Model'], rotation=45, ha='right', fontsize=10)
ax1.legend(loc='upper right', fontsize=10, framealpha=0.95)
ax1.set_ylim(0, 1.15)
ax1.set_facecolor('white')
ax1.grid(True, axis='y', alpha=0.3, color=colors['grid'])

# Add overfitting indicator - use red asterisk marker instead of emoji
overfitting_models = []
for i, (model, train_r2, test_r2) in enumerate(zip(results_df['Model'], results_df['Train R²'], results_df['Test R²'])):
    gap = train_r2 - test_r2
    if gap > 0.1:
        # Add red asterisk marker for overfitting warning
        ax1.plot(i, max(train_r2, test_r2) + 0.04, marker='*', markersize=15, 
                color=colors['warning'], markeredgecolor='white', markeredgewidth=0.5)
        overfitting_models.append(model)

ax1.axhline(y=0.8, color=colors['good'], linestyle='--', alpha=0.5, linewidth=1.5)
ax1.text(len(results_df)-0.5, 0.82, 'Good R² threshold', fontsize=9, color=colors['good'], ha='right')

# Add legend for overfitting marker
if overfitting_models:
    ax1.plot([], [], marker='*', markersize=12, color=colors['warning'], linestyle='None', 
             label='Overfitting risk')
    ax1.legend(loc='upper right', fontsize=9, framealpha=0.95)

# RMSE comparison - sorted by performance
ax2 = axes[1]
rmse_sorted = results_df.sort_values('Test RMSE')

# Create gradient colors based on RMSE (green=low, red=high)
norm_rmse = (rmse_sorted['Test RMSE'] - rmse_sorted['Test RMSE'].min()) / \
            (rmse_sorted['Test RMSE'].max() - rmse_sorted['Test RMSE'].min())
bar_colors = [plt.cm.RdYlGn_r(0.2 + 0.6 * v) for v in norm_rmse]

bars = ax2.barh(range(len(rmse_sorted)), rmse_sorted['Test RMSE'], 
                color=bar_colors, edgecolor='white', linewidth=1, alpha=0.85)

ax2.set_yticks(range(len(rmse_sorted)))
ax2.set_yticklabels(rmse_sorted['Model'], fontsize=10)
ax2.set_xlabel('Test RMSE (lower is better)', fontsize=12, color=colors['text'])
ax2.set_title('Test Set Error Comparison', fontsize=14, fontweight='bold', color=colors['text'])
ax2.set_facecolor('white')
ax2.grid(True, axis='x', alpha=0.3, color=colors['grid'])

# Add value labels
for i, (bar, rmse) in enumerate(zip(bars, rmse_sorted['Test RMSE'])):
    ax2.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2, 
             f'{rmse:.3f}', va='center', ha='left', fontsize=10, color=colors['text'])

ax2.set_xlim(0, rmse_sorted['Test RMSE'].max() * 1.2)

# Add annotation
ax2.text(0.98, 0.02, 'Green = better performance\nRed = worse performance', 
         transform=ax2.transAxes, fontsize=9, ha='right', va='bottom',
         color=colors['text'], style='italic',
         bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=colors['grid']))

plt.tight_layout()
plt.savefig('figures/04_model_comparison.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\nFigure saved to figures/04_model_comparison.png")
print(f"\n⚠️ Models with overfitting risk (Train-Test R² gap > 0.1): {', '.join(overfitting_models)}")

<cell_type>markdown</cell_type>## 3. Deep Dive: Random Forest

### 📖 Theory

<div style="background: rgba(16, 185, 129, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #10b981;">

**Random Forest** is an ensemble method that combines multiple decision trees to make robust predictions.

**Key Concepts:**

1. **Bagging (Bootstrap Aggregating)**: Each tree is trained on a random sample (with replacement) of the training data
2. **Feature Randomness**: At each split, only a random subset of features is considered
3. **Averaging**: Final prediction is the average of all tree predictions

**Why it works:**

$$\text{Var}(\bar{T}) = \frac{\sigma^2}{B} \cdot \left(1 + \frac{(B-1)\rho}{1}\right)$$

where $\rho$ is the correlation between trees. By decorrelating trees (through randomness), we reduce variance without increasing bias.

</div>

### Feature Importance

Random Forest provides built-in feature importance based on how much each feature decreases impurity:

$$I_j = \sum_{\text{trees}}\sum_{\text{nodes using } j} \frac{n_{\text{node}}}{n_{\text{total}}} \cdot \Delta_{\text{impurity}}$$

This tells us which features are most predictive for the target property.

In [ ]:
# Random Forest with feature importance
rf = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)

# Feature importance
importance = pd.DataFrame({
    'Feature': [f'Feature_{i}' for i in range(X.shape[1])],
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# Professional plot
fig, ax = plt.subplots(figsize=(12, 7), facecolor='white')

colors_palette = {
    'text': '#1e293b',
    'grid': '#e2e8f0'
}

# Top 10 features
top_n = 10
top_importance = importance.head(top_n)

# Gradient colors
cmap = plt.cm.viridis
bar_colors = [cmap(0.2 + 0.6 * (i / top_n)) for i in range(top_n)]

bars = ax.barh(range(top_n), top_importance['Importance'], 
               color=bar_colors, edgecolor='white', linewidth=1, alpha=0.85)

ax.set_yticks(range(top_n))
ax.set_yticklabels(top_importance['Feature'], fontsize=11, color=colors_palette['text'])
ax.set_xlabel('Feature Importance (Mean Decrease in Impurity)', fontsize=12, color=colors_palette['text'])
ax.set_title('Random Forest Feature Importance', fontsize=16, fontweight='bold', 
             color=colors_palette['text'], pad=15)
ax.invert_yaxis()
ax.set_facecolor('white')
ax.grid(True, axis='x', alpha=0.3, color=colors_palette['grid'])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add value labels
for bar, imp in zip(bars, top_importance['Importance']):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{imp:.3f}', va='center', ha='left', fontsize=10, color=colors_palette['text'])

ax.set_xlim(0, top_importance['Importance'].max() * 1.15)

# Highlight the features we know are important (0, 1, 2, 3, 4)
ax.text(0.98, 0.02, 'Features 0-4 were designed to be\npredictive in our synthetic data', 
        transform=ax.transAxes, fontsize=10, ha='right', va='bottom',
        color=colors_palette['text'], style='italic',
        bbox=dict(boxstyle='round,pad=0.4', facecolor='#f1f5f9', edgecolor=colors_palette['grid']))

plt.tight_layout()
plt.savefig('figures/04_rf_feature_importance.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

print("\n💡 Features 0 and 1 dominate because they have linear and quadratic relationships with the target!")
print("   Features 2, 3 contribute through their interaction term.")
print("\nFigure saved to figures/04_rf_feature_importance.png")

## 📝 Exercises

### Exercise: Tune Random Forest Hyperparameters

In [6]:
# Exercise: Try different n_estimators and max_depth
# What combination gives the best test R²?

# TODO: Create a grid search
# n_estimators_options = [50, 100, 200]
# max_depth_options = [5, 10, 15, 20]
# 
# for n_est in n_estimators_options:
#     for depth in max_depth_options:
#         rf = RandomForestRegressor(...)
#         ...

<cell_type>markdown</cell_type>---

## ✅ Module Summary

### Key Takeaways

<div style="background: rgba(99, 102, 241, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #6366f1;">

| Concept | Key Point |
|---------|-----------|
| **Linear models** | Good baselines, interpretable, but limited to linear relationships |
| **Regularization** | Ridge (L2) and Lasso (L1) prevent overfitting with many features |
| **Tree ensembles** | Random Forest and XGBoost capture non-linear patterns and interactions |
| **Overfitting** | Watch for large train-test R² gap (>0.1 is a warning sign) |
| **Feature importance** | Tree-based models reveal which features drive predictions |

</div>

### Model Selection Guidelines

```
Start with Linear Regression (baseline)
    ↓
If underfitting → Try Random Forest
    ↓
If need best performance → Tune XGBoost
    ↓
Always validate with cross-validation!
```

### Performance Metrics Reminder

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| R² | $1 - \frac{SS_{res}}{SS_{tot}}$ | Fraction of variance explained (higher is better) |
| MAE | $\frac{1}{n}\sum\|y - \hat{y}\|$ | Average absolute error (in original units) |
| RMSE | $\sqrt{\frac{1}{n}\sum(y - \hat{y})^2}$ | Penalizes large errors more than MAE |

### What's Next?

In **Module 5: Model Evaluation**, you'll learn to:
- Properly evaluate models with cross-validation
- Understand learning curves and validation curves
- Detect and prevent overfitting
- Choose between models systematically

---

**📚 Continue to Module 5:** [Model Evaluation](05_model_evaluation.ipynb)